In [11]:
import cv2
import numpy as np

In [ ]:
# load video path: sample_videos\Autonomous driving lane detection sample video 1.mp4

video_path = "sample_videos\Autonomous driving lane detection sample video 1.mp4"
cap = cv2.VideoCapture(video_path)


# change image frame to gray scale
def gray_scale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# apply gaussian blur to the image
def gaussian_blur(frame):
    return cv2.GaussianBlur(frame, (5, 5), 0)

# apply canny edge detection to the image
def canny_edge(frame):
    return cv2.Canny(frame, 50, 150)

# Apply a trapezoidal mask to the frame
def region_of_interest(frame):
    height, width = frame.shape[:2]
    mask = np.zeros_like(frame)

    # Define a trapezoid region of interest
    polygon = np.array([
        [
            (int(width * 0.2), height), # bottom left
            (int(width * 0.8), height), # bottom right
            (int(width * 0.65), int(height * 0.57)), # top right
            (int(width * 0.45), int(height * 0.57)) # top left
        ]
    ], dtype=np.int32)

    # Fill the mask with the polygon
    cv2.fillPoly(mask, polygon, 255)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)
    return masked_frame


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # convert to gray scale
    gray = gray_scale(frame)

    # apply gaussian blur
    blur = gaussian_blur(gray)

    # apply canny edge detection
    edge = canny_edge(blur)

    # Apply the region of interest mask
    masked_edge = region_of_interest(edge)

    # Show the video
    cv2.imshow("Edge detection", masked_edge)


    # video 60 fps, wait according to fps
    cv2.waitKey(int(1000/60))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break



